In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
text_spliter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap = 200, separators=["\n\n","\n"," ", ""])
loader = PyPDFLoader("Statefarm.pdf")
print("Loading documents...")
documents = loader.load()
print("Splitting documents...")
chunks = text_spliter.split_documents(documents)
print(f"Splitted the documents into {len(chunks)} chunks")
print("Creating vector store DB...")
vector_store = FAISS.from_documents(documents=chunks, embedding=embeddings)
print("Creating retriever...")
retriever = vector_store.as_retriever()

/Users/sunny/Desktop/RAG/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading documents...
Splitting documents...
Splitted the documents into 181 chunks
Creating vector store DB...
Creating retriever...


In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm_model = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ("system","""You are a knowledgeable car insurance company assistant specialized in answering questions about insurance policies, coverage, claims, and related services.
You must strictly follow these rules:
Only use information from the retrieved context (insurance policy documents, terms, and conditions).
Do not provide information that is not contained in the retrieved documents.
Provide clear, simple explanations about insurance policies, coverage options, claims processes, and related topics.
IF THE ANSWER IS NOT CONTAINED IN THE RETRIEVED TEXT, RESPOND WITH:
"The retrieved text does not contain enough information to answer this question. Please contact customer service for assistance."
When relevant:
Mention specific policy sections, coverage types, or terms from the provided context.
Summarize using plain language while maintaining accuracy about insurance terms and conditions.
Maintain a professional, helpful, and customer-service oriented tone.
Your job is to answer the user's question based solely on the insurance documentation provided in the context below."""),
("user","Question:{question}  \n Context: {context}")

])

chain = prompt | llm_model




In [5]:
query = "How do I file an auto insurance claim with State Farm?" 


# Retreiving the Documents
docs = retriever.invoke(query)

docs_as_string = "\n\n".join(doc.page_content for doc in docs)

# AGUMENTED + Generation

response = chain.invoke({
    "question": query,
    "context":  docs_as_string}
    )

# Print the response in a nicely formatted paragraph
print(response.content)

To file an auto insurance claim with State Farm, you should follow these steps as outlined in your policy:

1. **Report the Accident**: Promptly inform both the police and State Farm about the accident. If the accident involves an unknown driver or vehicle, you must report it as soon as possible.

2. **Keep Documentation**: If there are any lawsuits filed against the party responsible for the accident, you must immediately send State Farm a copy of all relevant lawsuit papers.

3. **Protect Property**: If there is property damage, you should take steps to protect the damaged property from further damage. Any reasonable expenses incurred for this must be reported to State Farm.

4. **Cooperate with Inspections**: Allow State Farm to inspect the damaged property before any repairs or disposal takes place, as well as during the repair process.

5. **Provide Additional Information**: Be prepared to provide any required information and cooperate with State Farm's requests, which may include